In [1]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
import cvxpy as cp
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read data
# data = pd.read_csv(HOME_DIR+'/data.csv')
data = pd.read_csv('data.csv')
data['month'] = data['month'].apply(lambda x: datetime.strptime(x, '%Y-%m')) #assume each date per month is the last day
data.set_index(['month'],inplace=True)
# No missing month

In [3]:
import McExperiment
from McExperiment import *
import importlib
importlib.reload(McExperiment)

<module 'McExperiment' from 'd:\\OneDriveLenovoD\\OneDrive\\A_Fall23\\FIN535\\HW2023\\HW3\\HW4_version2\\McExperiment.py'>

### Question 1
#### 1.1 Max Sharpe Ratio

In [6]:
### Theoretical Solution
from optimizer import * 
x5 = max_sharpe_opt1(data, 5)

optimal objective value 9.398639273665708
optimal var y [[111.06869882]
 [ 52.5030654 ]
 [ 45.25961111]
 [265.49589982]]
optimal allocation x [[0.25886057]
 [0.12236547]
 [0.10548362]
 [0.61877397]]


In [5]:
## Simulation Solution - Refer GridSearch.ipynb - 500 Grid
import pickle

opt_ratios = pickle.load(open('ratios_opt_Q1.pickle', 'rb'))

In [7]:
opt_ratios

,x1,x2,x3,x4
5Y_sharpe,0.19412821283008763,0.130897679253146,0.17178930119158897,0.6749741079167664
5Y_uti,0.7132951792394133,0.2484572146718702,1.8985635467053574,0.03824760608871647
5Y_prob,0.7132951792394133,0.2484572146718702,1.8985635467053574,0.03824760608871647
7Y_sharpe,0.19412821283008763,0.130897679253146,0.17178930119158897,0.6749741079167664
7Y_uti,0.7132951792394133,0.2484572146718702,1.8985635467053574,0.03824760608871647
7Y_prob,0.1534992420660981,0.8396523702258318,1.8978927120325957,0.006848387708070058


As we can see, the first row corresponds with our theoretical results, indicating that our methods are valid. We can either adopt theoretical or simulation results

In [8]:
# 5Y max Sharpe Strategy -------------------------------------------------------------------------------
# All value ar risk quoted in a return way (not in a loss way)
ratio1 = x5.reshape(-1).tolist()
print(ratio1)
SAVE = 40000
CAP = 50000
g = 0.05
MC = 119
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(2)

[0.25886056706172383, 0.12236546773821998, 0.10548362157645655, 0.6187739652000561]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,253634.10,16224.86,253634.10,0.0,1.47,0.07,0.10,0.09,-281699.85,-284555.46
7Y,386017.58,17628.64,386017.58,0.0,1.36,0.08,0.26,0.25,-148213.00,-148504.31
10Y,648093.09,20424.29,0.00,1.0,1.42,0.09,0.45,0.45,114892.36,112178.48


In [11]:
# 7Y max Sharpe Strategy -Use Simulation results ------------------------------------------------------------------------------
# ratio2 = x7.reshape(-1).tolist()
ratio2 = (opt_ratios.iloc[3,:]).to_list()
print(ratio2)
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio2,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(2)

[array(0.19412821), array(0.13089768), array(0.1717893), array(0.67497411)]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,253035.98,16170.05,253035.98,0.0,1.69,0.06,0.09,0.09,-279325.91,-280791.80
7Y,384189.40,17841.06,384189.40,0.0,1.56,0.07,0.24,0.23,-146530.17,-146834.08
10Y,642330.67,17356.73,0.00,1.0,1.61,0.08,0.43,0.43,110536.78,108921.92


##### Comments:

- Comment on Sharpe [TO DO]
- 7Y and 5Y same (numerical)

#### 1.2 Max Utility

In [12]:
# 5 year Max utility
ratio1 = (opt_ratios.iloc[1,:]).to_list()
print(ratio1)
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(5)

[array(0.71329518), array(0.24845721), array(1.89856355), array(0.03824761)]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,4.470467e+05,229344.51855,261400.77601,0.44538,1.18282,0.37020,-0.35962,-0.36413,-384537.67146,-389918.13168
7Y,8.800729e+05,434359.12938,364893.79629,0.68067,1.17354,0.42417,0.11224,0.10685,-206099.87901,-210938.91043
10Y,2.175540e+06,642952.53808,0.00000,1.00000,1.20511,0.48397,2.52515,2.51755,504842.11138,493677.82083


In [21]:
# 7Y max Utility Strategy -------------------------------------------------------------------------------
# Same as 5y

##### Comments

Say Expected Cap large [TO DO]
- Say why the results are same

#### 1.3 Max Probability of reaching goals

- Say Five years is the same
- Only display 7 years [TO DO]
[Extra comments: TO DO]

In [15]:
# 5 year Max prob
ratio1 = (opt_ratios.iloc[2,:]).to_list()
print(ratio1)
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(5)

[array(0.71329518), array(0.24845721), array(1.89856355), array(0.03824761)]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,4.470467e+05,229344.51855,261400.77601,0.44538,1.18282,0.37020,-0.35962,-0.36413,-384537.67146,-389918.13168
7Y,8.800729e+05,434359.12938,364893.79629,0.68067,1.17354,0.42417,0.11224,0.10685,-206099.87901,-210938.91043
10Y,2.175540e+06,642952.53808,0.00000,1.00000,1.20511,0.48397,2.52515,2.51755,504842.11138,493677.82083


In [16]:
# 7 year Max prob
ratio1 = (opt_ratios.iloc[5,:]).to_list()
print(ratio1)
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(5)

[array(0.15349924), array(0.83965237), array(1.89789271), array(0.00684839)]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,4.105402e+05,176099.23688,284064.42706,0.41176,0.91723,0.31155,-0.21109,-0.21657,-340959.52879,-346692.60133
7Y,7.609510e+05,310822.00297,380260.66712,0.72269,0.94663,0.34813,0.37398,0.35555,-167593.85586,-169984.15633
10Y,1.739684e+06,376413.72744,0.00000,1.00000,1.02679,0.35803,2.41942,2.34451,592183.19425,577911.73148


##### Comments
[to do]

### Question 2

#### 2.1 Max Sharpe Ratio

In [19]:
opt_ratios_q2 = pickle.load(open('ratios_opt_Q2.pickle', 'rb'))
opt_ratios_q2

,x1,x2,x3,x4
5Y_sharpe,0.23829047253115743,0.09224243267628252,0.16935657900502488,0.6694670947925601
5Y_uti,1.9817901066547705,-0.10873229426158981,1.8403095358621009,-0.8730578123931807
5Y_prob,0.36424434084062973,1.5736828795302582,1.9697562926494006,-0.9379272203708879
7Y_sharpe,0.23829047253115743,0.09224243267628252,0.16935657900502488,0.6694670947925601
7Y_uti,1.9817901066547705,-0.10873229426158981,1.8403095358621009,-0.8730578123931807
7Y_prob,0.40278169387381246,1.463911571241884,0.897755823229168,-0.8666932651156964


In [20]:
# 5Y & 7Y
# ratio1 = x5_2.reshape(-1).tolist()
ratio1 = (opt_ratios_q2.iloc[0,:]).to_list()
print(ratio1)
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(5)

[array(0.23829047), array(0.09224243), array(0.16935658), array(0.66946709)]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,253430.05761,18491.78308,253430.05761,0.0,1.53229,0.07356,0.07001,0.06966,-284558.32305,-286621.76016
7Y,386341.47219,21026.62069,386341.47219,0.0,1.42388,0.08511,0.23258,0.23121,-151262.69142,-151373.10598
10Y,650451.65845,21718.77039,0.00000,1.0,1.47457,0.09349,0.43365,0.43304,110557.62706,107686.00996


##### Comments
7 years same [to do]
They optimize fall to the same points

#### 2.2 Max Utility

In [21]:
# 5Y & 7Y
ratio1 = (opt_ratios_q2.iloc[1,:]).to_list()
print(ratio1)
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(5)

[array(1.98179011), array(-0.10873229), array(1.84030954), array(-0.87305781)]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,6.422002e+05,4.404846e+05,230875.10693,0.52101,0.93727,0.55157,-0.61945,-0.62188,-450554.84425,-453422.27715
7Y,1.484870e+06,9.867206e+05,341831.16807,0.68908,0.95963,0.62288,-0.20409,-0.22719,-298724.61551,-304134.67982
10Y,4.562911e+06,2.239065e+06,0.00000,1.00000,1.01639,0.70048,3.01123,2.97586,500737.21913,472032.66592


##### Comments [TO DO]

#### 2.3 Max Prob

In [22]:
# 5Y
ratio1 = (opt_ratios_q2.iloc[2,:]).to_list()
print(ratio1)
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(5)

[array(0.36424434), array(1.57368288), array(1.96975629), array(-0.93792722)]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,5.201568e+05,258343.96376,268458.14945,0.56303,0.94001,0.34678,-0.17034,-0.17904,-3.433289e+05,-3.461277e+05
7Y,1.043802e+06,498966.62462,417840.91098,0.78992,0.97621,0.39968,0.88815,0.86521,-1.432105e+05,-1.439645e+05
10Y,2.719683e+06,717789.18468,0.00000,1.00000,1.06231,0.40720,4.50132,4.27191,1.020615e+06,1.010890e+06


In [23]:
# 7Y
# Actually, in grid search, several grids share the same prob, we choose the one aligned with 5Y
ratio1 = (opt_ratios_q2.iloc[5,:]).to_list()
print(ratio1)
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(5)

[array(0.40278169), array(1.46391157), array(0.89775582), array(-0.86669327)]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,3.982785e+05,122429.25914,349538.38342,0.24370,0.94216,0.22571,0.06698,0.06264,-296301.47687,-300077.77214
7Y,7.181918e+05,201878.38809,433116.59541,0.78992,0.95870,0.25182,0.99481,0.93176,-117154.92276,-122900.02455
10Y,1.612176e+06,242285.28658,0.00000,1.00000,1.03515,0.25818,3.05672,2.95129,570482.01035,520843.52852


By allowing shorting for all assets and increasing risk free return rate to around 3.5%, we see that:
- Some assets are short-sold to increase sharpe ratio
- Allocation to bonds (TLT and T-bills) are reduced for both 5 and 7 years, perhaps due to the risk-adjusted returns for bonds are not as high as stocks
- The allocations with consideration of 5-year verses 7-year returns are relatively different, indicating that the returns  of those assets between the 5th and 7th year should undergone a dramatic shift in pattern


Indeed, these two year period corresponds to 2007-08-01 to 2009-08-01, the Great Recession. 
   - As economic conditions decline and investor uncertainty rises, more investors seek safety in high-quality fixed income investments, this drives bond prices up which pushes their returns down. 
   - Therefore, we seek to adjust our portfolio structures (e.g. shifting more towards stocks than bonds) when incorporating the changes that 2008 financial crisis has brought

### Question 3

To enhance the performance of the fixed mix strategy to imporve the probability of meeting the investor's goal, 
we should:
- initially, start with an optimal fixed mix strategy 
- throughout time, carefully monitor recent performance of all the assets invested as well as the investment returns
    - dynamically increase/decrease allocation to overperforming/underperforming assets
    - dynamically increase allocation to risker assets if we are suffuring from large drawdown to recover quickly 

In [4]:
# 5Y max Sharpe Strategy -------------------------------------------------------------------------------
# All value ar risk quoted in a return way (not in a loss way)
ratio1 = [3.11,4.619,6.49,-6.73]
print(ratio1)
SAVE = 40000
CAP = 50000
g = 0.05
MC = 119
# Get the analysis for end of 5Y/7Y/10Y 
Simu_table = Experiments(MC,data,CAP,SAVE,g,ratio1,years=[60,84,120],tax=0)
Summary_table = GetSummary(Simu_table)
Summary_table = Summary_table.fillna(0)
Summary_table.round(2)

[3.11, 4.619, 6.49, -6.73]


,Expected Cap,Std Cap,Exp Cap < 500k,Prob >= 500k,Expected Annual_sharpe,Expected MDD,ret VaR,ret CVaR,GaR,CGaR
5Y,1.850513e+07,2.577320e+07,136345.62,0.72,1.16,0.75,-1.00,-1.00,-498845.08,-499011.14
7Y,1.126849e+08,1.441969e+08,189210.48,0.82,1.18,0.83,-0.96,-0.96,-489823.38,-490525.50
10Y,8.929770e+08,1.091271e+09,0.00,1.00,1.27,0.89,1.38,1.07,5206607.65,4612085.97
